In [ ]:
import os
import numpy as np
import pandas as pd
import cooler
from sklearn import metrics
from sklearn.decomposition import PCA
import math
import sklearn.metrics as lunkuo


def getResultList(resultPath,chr):
    list = os.listdir(resultPath)
    namelist =[]
    boundarylist = []
    for file in list:
        namelist.append(file.replace("_Result.txt",""))
        mare = open(resultPath+"/"+file)
        mathlist = []
        for lin in mare:
            a = lin.strip().split("-")
            first = a[0]
            mathlist.append(first)
        boundarylist.append(mathlist)
        mare.close()

    # for index in range(len(namelist)):
    #     ff = 0
    #     count = 0
    #     for val in boundarylist[index]:
    #         aa = int(val)-ff
    #         count = count +aa
    #         ff = int(val)
    #     count = int(count / len(boundarylist[index]))
    #
    #     print('{0}\t{1}\t{2}'.format(namelist[index],len(boundarylist[index]),count))

    return namelist, boundarylist


def getsignList(fileParh,chr):
    difile = open(fileParh)
    dilist = []
    for lin in difile:
        a = lin.strip().split("\t")
        dilist.append(float(a[3]))
    difile.close()
    return dilist


def get_CTCF_makerlist(markpath, chrnum, mat_len, resulution):
    ctcf_data = pd.read_csv(markpath, sep='\t', header=None)
    ctcf_begin = np.array(np.ceil(np.array(ctcf_data[ctcf_data[0] == 'chr{0}'.format(chrnum)][1])/resulution),dtype=np.int)
    ctcf_end = np.array(np.ceil(np.array(ctcf_data[ctcf_data[0] == 'chr{0}'.format(chrnum)][2])/resulution),dtype=np.int)
    peak_list = np.array([0] * mat_len)
    for index in range(len(ctcf_begin)):
        bb = ctcf_begin[index]
        ee = ctcf_end[index]+1
        peak_list[bb:ee] = peak_list[bb:ee] + 1
    return peak_list


def biomakercompare(namelist,boundarylist,peaklist,clength):
    picname = ""
    picvalue = [""] * (clength * 2 + 1)
    for index in range(len(namelist)):
        print(namelist[index])
        picname= picname + namelist[index]+"\t"
        count = 0
        final = [0]*(clength*2+1)
        for bin in boundarylist[index]:
            location = int(bin)
            if location - clength > -1 and location + clength < 4864:
                ll = peaklist[location-clength:location+clength+1]
                for index in range(len(ll)):
                    final[index] = final[index] + int(ll[index])
                count += 1
        for index in range(len(final)):
            print(final[index]/count)
            picvalue[index] = picvalue[index] +"{0}\t".format(final[index]/count)
    print(picname)
    for lin in picvalue:
        print(lin)


def getdiffResult(resultPath,chr):
    list = os.listdir(resultPath)
    namelist =[]
    boundarylist = []
    for file in list:
        namelist.append(file.replace(".txt",""))
        mare = open(resultPath+"/"+file)
        mathlist = []
        for lin in mare:
            a = lin.strip().split("-")
            first = a[0]
            secend = a[1]
            mathlist.append(first)
        boundarylist.append(mathlist)
        mare.close()
    return namelist, boundarylist


def numbercompare(namelist,boundarylist):
    # size比较
    relength =[]
    for linst in boundarylist:
        re = []
        first = linst[0]
        for index in range(1, len(linst)):
            re.append(int(linst[index]) - int(first))
            first = linst[index]
        relength.append(re)
    f = open("./sizecompare.txt","w")
    for index in range(len(relength)):
        for a in relength[index]:
            f.write("{0} {1}\n".format(a,namelist[index]))
    f.close()
    # 数量比较
    for index in range(len(relength)):
        print("{0}\t{1}".format(namelist[index],len(boundarylist[index])))


def boundarysame(namelist,boundarylist):
    base = namelist[5]
    baselist = boundarylist[5]

    print("base\taim\tbasevalue\taimvalue\tsame ")

    alllist = []
    allname = []
    for index in range(len(namelist)):
        count = 0
        baseonly = []
        aimonly = []
        common = []

        for value in boundarylist[index]:
            if value in baselist:
                common.append(value)
                count = count+1
            else:
                aimonly.append(value)
        for value in baselist:
            if value not in boundarylist[index]:
                baseonly.append(value)

        allname.append('{0}-1'.format(namelist[index]))
        allname.append('{0}-2'.format(namelist[index]))
        allname.append('{0}-3'.format(namelist[index]))
        alllist.append(baseonly)
        alllist.append(aimonly)
        alllist.append(common)

        print("{0}\t{1}\t{2}\t{3}\t{4}".format(base, namelist[index], len(baselist), len(boundarylist[index]), count))
    return allname,alllist


def getTADCount(namelist,boundarylist):
    b = np.loadtxt("./data/ice.txt")
    f = open('./avcount','w')
    matrix2 = np.triu(b)
    df_iced_mat = pd.DataFrame(matrix2)

    alllist =[]
    for index in range(len(namelist)):
        # 每一个方法
        meth = boundarylist[index]
        first = 0
        cTADlist =[]
        for value in meth:
            # 每一个TAD
            secend = int(value)
            sli = df_iced_mat.iloc[first:secend, first:secend]
            count = 0
            length = secend-first
            for i in sli.columns:
                count = count+sli[i].sum()
            if length >0:
                avge = count/length
                cTADlist.append(avge)
        alllist.append(cTADlist)

    for index in range(len(namelist)):
        ll = alllist[index]
        for value in ll :
            f.write("{0} {1}\n".format(value,namelist[index]))
    f.close()

    for lll in alllist:
        print('-------------')
        print(len(lll))
        print(sum(lll)/len(lll))


def biomakercompare2(namelist,boundarylist,peaklist,clength):
    picname = ""
    picvalue = [""] * (clength * 2 + 1)
    bigest = []
    for index in range(len(namelist)):
        aaaa = namelist[index]
        picname= picname + namelist[index]+"\t"
        count = 0
        final = [0]*(clength*2+1)
        center = []
        zuoyou = []
        for bin in boundarylist[index]:
            location = int(bin)
            if location - clength > -1 and location + clength < 4864:
                ll = peaklist[location-clength:location+clength+1]
                for index in range(len(ll)):
                    final[index] = final[index] + int(ll[index])
                count += 1
        # print(final)
        print(aaaa)
        for index in range(len(final)):
            print(final[index]/count)
            picvalue[index] = picvalue[index] +"{0}\t".format(final[index]/count)

        bigest.append(max(final)/count)

        print(bigest)


def boundaryScore(namelist, boundarylist, bdsocrepath):
    bdsc = np.loadtxt(bdsocrepath)
    for index in range(0,len(namelist)):
        boundary = boundarylist[index]
        sumlist = []
        for aa in boundary:
            sumlist.append(bdsc[eval(aa)-1])
        np.savetxt('./a1.6/{0}_box.txt'.format(namelist[index]),sumlist,fmt='%1.3f')
        sumvalue = sum(sumlist)
        sumvalue = sumvalue/len(boundary)
        print("{0}\t{1}".format( namelist[index], sumvalue))


def figer3():
    v1 = open('./fomat_result/Rao2014-GM12878-DpnII-allreps-filtered-50kb/chr2/FinalResult/McNodeGraph_Result.txt')
    baselist =[]
    for lin in v1:
        a = lin.strip().split('-')
        baselist.append(int(a[0]))
    v1.close()

    print(baselist)
    f = open('./10sum')
    ci = []
    for vaa in f:
        svalue = vaa.strip()
        value = float(svalue)
        ci.append(value)
    f.close()
    print(ci)
    namelist = []
    namelist.append('good')
    namelist.append('bad')
    boulist = []
    goodlist = []
    badlist = []
    for index in baselist:
        if ci[index] < 0.14 and ci[index] > 0.05:
            goodlist.append(index)
            print(index)
        else:
            badlist.append(index)
    print(len(goodlist))
    boulist.append(goodlist)
    boulist.append(badlist)

    return namelist, boulist


def get_mat_length(cool_path):
    coolfile = cooler.Cooler(cool_path)
    length_list =[]
    for index in range(1,24):
        if index == 23:
            wt = coolfile.matrix(balance=False).fetch('chrX')
            b = wt[:]
            length_list.append(np.shape(b)[0])
        else:
            wt = coolfile.matrix(balance=False).fetch('chr{0}'.format(index))
            b = wt[:]
            length_list.append(np.shape(b)[0])
    return length_list


def di_result_format(result_path,celline):
    result_list = []
    for index in range(1,23):
        filepath = "{0}/{1}chr{2}finaldomaincalls".format(result_path, celline, index)
        data = pd.read_csv(filepath, header=None, sep='\t')
        re = data.loc[:,1]/50000
        re = np.array(re, dtype=np.int).tolist()
        result_list.append(re)
        print('lens of di:{0}'.format(len(re)))
    return result_list


def di_25kb_result_format(result_path,celline):
    result_list = []
    for index in range(1,23):
        # GM12878_25kb_chr10finaldomaincalls
        filepath = "{0}/{1}_25kb_chr{2}finaldomaincalls".format(result_path, celline, index)
        print(filepath)
        data = pd.read_csv(filepath, header=None, sep='\t')
        re = data.loc[:,1]/25000
        re = np.array(re, dtype=np.int).tolist()
        result_list.append(re)
        print('lens of di:{0}'.format(len(re)))
    return result_list


def di_100kb_result_format(result_path,celline):
    result_list = []
    for index in range(1,23):
        # GM12878_25kb_chr10finaldomaincalls
        filepath = "{0}/{1}_100kb_chr{2}finaldomaincalls".format(result_path, celline, index)
        print(filepath)
        data = pd.read_csv(filepath, header=None, sep='\t')
        re = data.loc[:,1]/100000
        re = np.array(re, dtype=np.int).tolist()
        result_list.append(re)
        print('lens of di:{0}'.format(len(re)))
    return result_list


def down_samping_di_result_format(result_path, cellines):
    all_re_list = {}
    for cell in cellines:
        result_list = []
        for index in range(1,23):
            # GM12878_HUVECchr9finaldomaincalls
            filepath = "{0}/GM12878_{1}chr{2}finaldomaincalls".format(result_path, cell, index)
            data = pd.read_csv(filepath, header=None, sep='\t')
            re = data.loc[:,1]/50000
            re = np.array(re, dtype=np.int).tolist()
            result_list.append(re)
        all_re_list[cell] = result_list
    return all_re_list


def down_samping_di_result_format2(result_path, cellines):
    result_list = []
    for index in range(1, 23):
        # GM12878_HUVECchr9finaldomaincalls
        filepath = "{0}/GM12878_{1}chr{2}finaldomaincalls".format(result_path, cellines, index)
        data = pd.read_csv(filepath, header=None, sep='\t')
        re = data.loc[:, 1] / 50000
        re = np.array(re, dtype=np.int).tolist()
        result_list.append(re)
    return result_list


def insulation_result_format(result_path,celline):
    result_list = []
    for index in range(1,23):
        filepath = "{0}/{1}{2}_insulation.is500001.ids200001.insulation.boundaries".format(result_path,celline,index)
        data = pd.read_csv(filepath,sep='\t')
        strs = data['header.1']
        result = []
        for st in strs:
            spls = int(st.split('|')[0].replace('bin', ''))
            result.append(spls)
        print('lens of is:{0}'.format(len(result)))
        result_list.append(result)
    return result_list

def insulation_25kb_result_format(result_path,celline):
    result_list = []
    for index in range(1,23):
        filepath = "{0}/{1}{2}_insulation.is1000001.ids200001.insulation.boundaries".format(result_path,celline,index)
        data = pd.read_csv(filepath,sep='\t')
        strs = data['header.1']
        result = []
        for st in strs:
            spls = int(st.split('|')[0].replace('bin', ''))
            result.append(spls)
        print('lens of is:{0}'.format(len(result)))
        result_list.append(result)
    return result_list


def insulation_100kb_result_format(result_path,celline):
    result_list = []
    for index in range(1,23):
        filepath = "{0}/{1}{2}_insulation.is500001.ids200001.insulation.boundaries".format(result_path,celline,index)
        data = pd.read_csv(filepath,sep='\t')
        strs = data['header.1']
        result = []
        for st in strs:
            spls = int(st.split('|')[0].replace('bin', ''))
            result.append(spls)
        print('lens of is:{0}'.format(len(result)))
        result_list.append(result)
    return result_list



def down_samping_insulation_result_format(result_path,cellines):
    all_re_list = {}
    for cell in cellines:
        result_list = []
        for index in range(1,23):
            filepath = "{0}/GM12878_{1}_{2}_insulation.is500001.ids200001.insulation.boundaries".format(result_path,cell,index)
            data = pd.read_csv(filepath,sep='\t')
            strs = data['header.1']
            result = []
            for st in strs:
                spls = int(st.split('|')[0].replace('bin', ''))
                result.append(spls)
            print('lens of is:{0}'.format(len(result)))
            result_list.append(result)
        all_re_list[cell] = result_list
    return all_re_list


def down_samping_insulation_result_format2(result_path,cellines):
    result_list = []
    for index in range(1, 23):
        filepath = "{0}/GM12878_{1}_{2}_insulation.is500001.ids200001.insulation.boundaries".format(result_path, cellines,
                                                                                                    index)
        data = pd.read_csv(filepath, sep='\t')
        strs = data['header.1']
        result = []
        for st in strs:
            spls = int(st.split('|')[0].replace('bin', ''))
            result.append(spls)
        print('lens of is:{0}'.format(len(result)))
        result_list.append(result)
    return result_list


def topdom_result_format(result_path,celline):
    print('run')
    result_list = []
    for index in range(1,23):
        filepath = "{0}/{1}{2}_topdomResult.domain".format(result_path, celline, index)
        data = pd.read_csv(filepath, sep='\t')
        topre = np.array(data['from.id'])
        result_list.append(topre)
        print('lens of topdom:{0}'.format(len(topre)))
    return result_list


def topdom_25kb_result_format(result_path, celline):
    print('run')
    result_list = []
    for index in range(1,23):
        filepath = "{0}/{1}_25kb_chr{2}_topdomResult.domain".format(result_path, celline, index)
        data = pd.read_csv(filepath, sep='\t')
        topre = np.array(data['from.id'])
        result_list.append(topre)
        print('lens of topdom:{0}'.format(len(topre)))
    return result_list


def topdom_100kb_result_format(result_path, celline):
    print('run')
    result_list = []
    for index in range(1,23):
        filepath = "{0}/{1}_100kb_chr{2}_topdomResult.domain".format(result_path, celline, index)
        data = pd.read_csv(filepath, sep='\t')
        topre = np.array(data['from.id'])
        result_list.append(topre)
        print('lens of topdom:{0}'.format(len(topre)))
    return result_list


def down_samping_topdom_result_format(result_path,cellines):
    print('run')
    all_re_list = {}
    for cell in cellines:
        result_list = []
        for index in range(1,23):
            # GM12878_NHEK_14_topdomResult.domain
            filepath = "{0}/GM12878_{1}_{2}_topdomResult.domain".format(result_path, cell, index)
            data = pd.read_csv(filepath, sep='\t')
            topre = np.array(data['from.id'])
            result_list.append(topre)
            print('lens of topdom:{0}'.format(len(topre)))
        all_re_list[cell] = result_list
    return all_re_list


def down_samping_topdom_result_format2(result_path,cellin):
    print('run')
    result_list = []
    for index in range(1, 23):
        # GM12878_NHEK_14_topdomResult.domain
        filepath = "{0}/GM12878_{1}_{2}_topdomResult.domain".format(result_path, cellin, index)
        data = pd.read_csv(filepath, sep='\t')
        topre = np.array(data['from.id'])
        result_list.append(topre)
        print('lens of topdom:{0}'.format(len(topre)))
    return result_list


def mctad_result_format(result_path, threshold):
    print('run')
    result_list = []
    for index in range(1,23):
        hold_list = []
        filepath = "{0}/chr{1}_min_score.txt".format(result_path, index)
        valuepath = "{0}/chr{1}_min_value.txt".format(result_path, index)
        reloca = np.loadtxt(filepath, dtype=np.int)
        rescore = np.loadtxt(valuepath)
        for index in range(len(rescore)):
            if rescore[index] < threshold:
                hold_list.append(reloca[index])
        print('lens of mctad:{0}'.format(len(hold_list)))
        result_list.append(hold_list)

    return result_list

def Emtad_result_format(result_path):
    print('run')
    result_list = []
    for index in range(1,23):
        filepath = "{0}/chr{1}.txt".format(result_path, index)
        reloca = np.loadtxt(filepath, dtype=np.int)
        result_list.append(reloca[0:-1])

    return result_list


def mctad_25kb_result_format(result_path, threshold, celline):
    print('run')
    result_list = []
    for index in range(1,23):
        hold_list = []
        filepath = "{0}/{1}_chr{2}_min_score.txt".format(result_path,celline, index)
        valuepath = "{0}/{1}_chr{2}_min_value.txt".format(result_path, celline, index)
        reloca = np.loadtxt(filepath, dtype=np.int)
        rescore = np.loadtxt(valuepath)
        for index in range(len(rescore)):
            if rescore[index] < threshold:
                hold_list.append(reloca[index])
        print('lens of mctad:{0}'.format(len(hold_list)))
        result_list.append(np.array(hold_list))

    return result_list


def mctad_100kb_result_format(result_path, threshold, celline):
    print('run')
    result_list = []
    for index in range(1,23):
        hold_list = []
        filepath = "{0}/{1}_chr{2}_min_score.txt".format(result_path,celline, index)
        valuepath = "{0}/{1}_chr{2}_min_value.txt".format(result_path, celline, index)
        reloca = np.loadtxt(filepath, dtype=np.int)
        rescore = np.loadtxt(valuepath)
        for index in range(len(rescore)):
            if rescore[index] < threshold:
                hold_list.append(reloca[index])
        print('lens of mctad:{0}'.format(len(hold_list)))
        result_list.append(np.array(hold_list))

    return result_list


def down_samping_mctad_result_format(result_path, threshold, cellines):
    print('run')
    all_cellines_re = {}
    for cell in cellines:
        result_list = []
        for index in range(1,23):
            hold_list = []
            filepath = "{0}/{1}_chr{2}_min_score.txt".format(result_path, cell, index)
            valuepath = "{0}/{1}_chr{2}_min_value.txt".format(result_path, cell, index)
            reloca = np.loadtxt(filepath, dtype=np.int)
            rescore = np.loadtxt(valuepath)
            for index in range(len(rescore)):
                if rescore[index] < threshold:
                    hold_list.append(reloca[index])
            print('lens of mctad:{0}'.format(len(hold_list)))
            result_list.append(hold_list)
        all_cellines_re[cell] = result_list

    return all_cellines_re


def down_samping_mctad_result_format2(result_path, threshold, cellines):
    print('run')
    result_list = []
    for index in range(1, 23):
        hold_list = []
        filepath = "{0}/{1}_chr{2}_min_score.txt".format(result_path, cellines, index)
        valuepath = "{0}/{1}_chr{2}_min_value.txt".format(result_path, cellines, index)
        reloca = np.loadtxt(filepath, dtype=np.int)
        rescore = np.loadtxt(valuepath)
        for index in range(len(rescore)):
            if rescore[index] < threshold:
                hold_list.append(reloca[index])
        print('lens of mctad:{0}'.format(len(hold_list)))
        result_list.append(hold_list)

    return result_list


def peak_result(peakfilepath):
    peak_lists = []
    for index in range(1,23):
        peaks = getbiomakerlist(peakfilepath, "chr{0}".format(index))
        peak_lists.append(peaks)
    return peak_lists


def compare_biomaker_results(results_list,biomaker_list,length,mat_len_list):
    bio_result_list = []
    for id in range(len(results_list)):
        mat_length = mat_len_list[id]
        bio_result =[0]*11
        resu = results_list[id]
        bio = biomaker_list[id]
        count = 0
        for locaa in resu:
            loca = int(locaa)
            if loca > length and mat_length - length > loca:
                reloca = int(loca - 1)
                rr = np.array(bio[reloca-length:reloca+length+1])
                bio_result = bio_result+rr
                count +=1
        bio_result_list.append(bio_result/count)
    for bb in bio_result_list:
        print(np.max(bb))
    return bio_result_list


def compare_insulation_results(results_list,biomaker_list,length,mat_len_list):
    bio_result_list = []
    for id in range(len(results_list)):
        mat_length = mat_len_list[id]
        bio_result =[0]*11
        resu = results_list[id]
        bio = biomaker_list[id]
        count = 0
        for locaa in resu:
            loca = int(locaa)
            if loca > length and mat_length - length > loca:
                reloca = int(loca - 1)
                rr = np.array(bio[reloca-length:reloca+length+1])
                bio_result = bio_result+rr
                count +=1
        bio_result_list.append(bio_result/count)
    for bb in bio_result_list:
        print(np.min(bb))
    return bio_result_list


def compare_insulation_results2(results_list,biomaker_list):
    bio_result_list = np.zeros(10)
    for id in range(len(results_list)):
        all_value = np.zeros(10)
        resu = results_list[id]
        insu = biomaker_list[id]
        for location in range(3,len(resu)-3):
            invalue = insu[resu[location]-5:resu[location]+5]
            all_value  = all_value + invalue
        all_value = all_value/(len(resu)-6)
        bio_result_list = bio_result_list + all_value

    for vv in bio_result_list:
        print(vv)
    print('-----')
    return bio_result_list


def compare_CI_with_top_boundary(results_list,biomaker_list,number):
    bio_result_list = []
    for id in range(len(results_list)):
        ci_value_list = []
        resu = results_list[id]
        insu = biomaker_list[id]
        boundary_number = number[id]
        for location in resu:
            invalue = insu[location]
            ci_value_list.append(invalue)
        ci_value_list = np.array(ci_value_list, dtype=np.float)
        ci_value_list.sort()

        all_value = 0
        for index in range(boundary_number):
            all_value += ci_value_list[index]

        all_value = all_value/boundary_number
        bio_result_list.append(all_value)
    x = np.array(bio_result_list)
    return x


def get_lunkuoxishu(mat, results, mat_length):
    ice_mat = mat
    re = np.where(np.all(ice_mat == 0, axis=0))

    dele_mat = np.delete(ice_mat, re, axis=0)
    dele_mat = np.delete(dele_mat, re, axis=1)


    cl = 0
    begin = 0
    labels = results
    labels = np.append(labels,mat_length)
    lable_list=[]
    for rande in labels:
        for index in range(begin,rande):
            lable_list.append(cl)
        begin = rande
        cl += 1

    relist = np.array(lable_list)
    finale = np.delete(relist, re)
    # hic_mat = np.loadtxt(mat_path)[0:1600,0:1600]
    # hic_mat = original_matrix_to_ice_matrix(hic_mat)
    # print(np.shape(hic_mat))
    # print(np.shape(lable_list))
    # print(labels)
    # print(lable_list)

    pca = PCA(n_components=300)
    pca_dele_mat = pca.fit_transform(dele_mat)
    value = metrics.silhouette_score(pca_dele_mat, finale, metric='cosine')
    # ['euclidean', 'l2', 'l1', 'manhattan', 'cityblock', 'braycurtis', 'canberra', 'chebyshev', 'correlation', 'cosine',
    #  'dice', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao',
    #  'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule', 'wminkowski', 'nan_euclidean', 'haversine']

    print(value)


def get_insulation_score_list(folder_path):
    insulation_score_list = []
    for index in range(1,23):
        file_path = "{0}/GM12878{1}_insulation.is500001.ids200001.insulation".format(folder_path, index)
        ins = pd.read_csv(file_path, sep='\t')
        ins = ins.fillna(0)
        re = np.array(ins['insulationScore'])
        re[re > 10] = 10
        re[re < -10] = -10
        insulation_score_list.append(re)
    return insulation_score_list


def compare_celline_lunkuoxishu(mat_folder, results, mat_length_list,celline):
    for index in range(len(results)):
        mat = np.loadtxt("{0}/{1}{2}".format(mat_folder,celline,index+1))
        get_lunkuoxishu(mat, results[index], mat_length_list[index])


def get_CI_list(mat_folder, celline):
    for index in range(1,23):
        mat_path = '{0}/{1}{2}'.format(mat_folder,celline,index)
        mat = np.loadtxt(mat_path)
        binrange = 5
        bias = 0.0001
        emdshape = np.shape(mat)
        emdCount = np.zeros(emdshape[0])
        matrix = mat
        for index in range(binrange, emdshape[0] - binrange):
            point = index
            uavalue = 0
            davalue = 0
            for i in range(1, 3):
                ua = point - i
                uavalue = uavalue + np.sum(matrix[ua - 4:ua, ua + 1:ua + 5]) / 16
            cvalue = np.sum(matrix[point - 4:point, point + 1:point + 5]) / 16
            for i in range(1, 3):
                na = point + i
                davalue = davalue + np.sum(matrix[na - 4:na, na + 1:na + 5]) / 16
            sum_value = davalue + uavalue + bias
            sum_value = cvalue / sum_value
            if sum_value < 0:
                sum_value = 0
            emdCount[index] = sum_value
        np.savetxt('{0}_CIvalue2'.format(mat_path), emdCount, fmt='%.4f')


def load_CI_list(mat_folder, celline):
    CI_list =[]
    for index in range(1,23):
        cifile = '{0}/{1}{2}_CIvalue'.format(mat_folder, celline, index)
        cci = np.loadtxt(cifile)
        CI_list.append(cci)
    return CI_list


def load_CI_list2(mat_folder, celline):
    CI_list =[]
    for index in range(1,23):
        cifile = '{0}/{1}{2}_CIvalue2'.format(mat_folder, celline, index)
        cci = np.loadtxt(cifile)
        CI_list.append(cci)
    return CI_list


def compare_boundary_score(all_result_list, bin_range):
    mctad_result_list = all_result_list[0]
    topdom_result_list = all_result_list[1]
    di_result_list = all_result_list[2]
    is_result_list = all_result_list[3]

    all_compare_result = []
    for method_index in range(len(all_result_list)):
        method_chrs_result = []
        for chrnum in range(22):
            bijiao = all_result_list[method_index][chrnum]
            bijiao_result = []
            for bin in bijiao:
                loc_score = 0
                range_list = [bin]
                for range_index in range(1,bin_range+1):
                    range_list.append(bin - range_index)
                    range_list.append(bin + range_index)
                if len(list(set(range_list) & set(mctad_result_list[chrnum]))) != 0:
                    loc_score += 1
                if len(list(set(range_list) & set(topdom_result_list[chrnum]))) != 0:
                    loc_score += 1
                if len(list(set(range_list) & set(di_result_list[chrnum]))) != 0:
                    loc_score += 1
                if len(list(set(range_list) & set(is_result_list[chrnum]))) != 0:
                    loc_score += 1
                bijiao_result.append(loc_score)
            method_chrs_result.append(bijiao_result)
        all_compare_result.append(method_chrs_result)
    return all_compare_result


def get_venn_data(all_result_list,celline):
    final_venn_list = []
    for celline_result in all_result_list:
        celline_result_list = []
        for chrnum in range(len(celline_result)):
            for loca in celline_result[chrnum]:
                celline_result_list.append('{0}-{1}'.format(chrnum, loca))
        final_venn_list.append(celline_result_list)

    print(len(final_venn_list))

    mydata = {
        "MCTad": final_venn_list[0],
        "TopDom": final_venn_list[1],
        "Directionality":final_venn_list[2],
        "Insulation":final_venn_list[3]
    }

    locdata = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in mydata.items()]))
    locdata.to_csv('./{0}_demoloc.txt'.format(celline),sep='\t',index=False)


def compare_bdscore_marker(scre, CI_list,celline):
    qurelist = []
    methlist =['MCTad', 'TopDom', 'Directionality', 'Insulation']
    mctad_qury = scre.query('MCTad== True & TopDom == False  & Directionality == False & Insulation == False')
    topDom_qury = scre.query('MCTad== False & TopDom == True  & Directionality == False & Insulation == False')
    di_qury = scre.query('MCTad== False & TopDom == False  & Directionality == True & Insulation == False')
    insu_qury = scre.query('MCTad== False & TopDom == False  & Directionality == False & Insulation == True')
    qurelist.append(mctad_qury)
    qurelist.append(topDom_qury)
    qurelist.append(di_qury)
    qurelist.append(insu_qury)
    all_qure_value = []
    all_name_list =[]
    for qu_index in range(len(qurelist)):
        th_qury = qurelist[qu_index]
        method_name = methlist[qu_index]
        idarr = np.array(th_qury['id'])
        for value in idarr:
            spvalue = value.split('-')
            chrnum = int(spvalue[0])
            ciloc = int(spvalue[1])
            civalue = CI_list[chrnum][ciloc]
            all_qure_value.append(civalue)
            all_name_list.append(method_name)
            # print("{0}\t{1}\t{2}".format(chrnum,ciloc,civalue))
    x = np.array(all_qure_value)
    Min = np.min(x)
    Max = np.max(x)
    x = 1 - ((x - Min) / (Max - Min))
    print("{0}-{1}".format(len(x),len(all_name_list)))
    for index in range(len(x)):
        print("{0}\t{1}".format(x[index],all_name_list[index]))


def get_overlap_length(st1, ed1, st2, ed2):
    if ed1 <= st2 or st1 >= ed2:
        overlap = 0
    elif st2 >= st1 and st2 <= ed1 and ed2 > ed1:
        overlap = ed1 - st2
    elif st1 >= st2 and st1 <= ed2 and ed1 > ed2:
        overlap = ed2 - st1
    elif st1 >= st2 and ed1 <= ed2:
        overlap = ed1 - st1
    elif st2 >= st1 and ed2 <= ed1:
        overlap = ed2 - st2
    return overlap


def trans_to_dataframe(re_list):
    chr1 = np.array(re_list)
    redik = {}
    chr1 = chr1*50000
    redik['start'] = list(chr1[:-1])
    redik['end'] = list(chr1[1:])
    df = pd.DataFrame.from_dict(redik)
    return df


def calculate_MoC_of_TAD_domain(df_TAD_m1, df_TAD_m2):
    record = []
    moc_value = 0
    n1 = len(df_TAD_m1)
    n2 = len(df_TAD_m2)
    for i in range(len(df_TAD_m1)):
        st1 = df_TAD_m1['start'][i]
        ed1 = df_TAD_m1['end'][i]
        length1 = int((ed1 - st1) / 1000)
        for j in range(len(df_TAD_m2)):
            st2 = df_TAD_m2['start'][j]
            ed2 = df_TAD_m2['end'][j]
            length2 = int((ed2 - st2) / 1000)
            overlap = get_overlap_length(st1, ed1, st2, ed2)
            overlap = int(overlap / 1000)
            if length1 != 0 and length2 !=0:
                add_value = int(overlap)**2 / (length1 * length2)
            else:
                add_value = 0
            moc_value += add_value
            record.append(add_value)
    MoC_value = (1/(np.sqrt(n1*n2) - 1)) * (moc_value - 1)
    return MoC_value


def rand_index(y_true, y_pred):
    n = min(len(y_pred), len(y_true))
    a, b = 0, 0
    for i in range(n):
        for j in range(i + 1, n):
            if (y_true[i] == y_true[j]) & (y_pred[i] == y_pred[j]):
                a += 1
            elif (y_true[i] != y_true[j]) & (y_pred[i] != y_pred[j]):
                b += 1
            else:
                pass
    RI = (a + b) / (n * (n - 1) / 2)
    return RI


def list_to_labes(list):
    labe = 0
    begin = list[0]
    lable_list = []
    for rande in list:
        for index in range(begin,rande):
            lable_list.append(labe)
        begin = rande
        labe +=1
    return lable_list


def MI_list_to_labes(numlist,matlength):
    labe = 0
    begin = 0
    numlist = list(numlist)
    numlist.append(matlength)
    lable_list = []
    for rande in numlist:
        for index in range(begin,rande):
            lable_list.append(labe)
        begin = rande
        labe +=1
    return lable_list


def get_rand_index(first_re_list,secend_re_list):
    for index in range(len(first_re_list)):
        fire = list_to_labes(first_re_list[index])
        sere = list_to_labes(secend_re_list[index])
        value = rand_index(fire,sere)
        print(value)


def get_mutual_information(first_re_list,secend_re_list,mat_lens):
    for index in range(len(first_re_list)):
        lens = mat_lens[index]
        fire = MI_list_to_labes(first_re_list[index],lens)
        sere = MI_list_to_labes(secend_re_list[index],lens)
        value = metrics.mutual_info_score(fire, sere)
        print(value)



if __name__ == '__main__':
    print('run')
    mat_len_list = np.loadtxt('./hunmen_gens_length', dtype=np.int)
    # mat_len_list = get_mat_length('/mnt/disk1/duanran/Rao2014-GM12878-DpnII-allreps-filtered.50kb.cool')
    print(mat_len_list)
    di_result_list = di_result_format('./perl_scripts/GM12878result','GM12878')
    peak_list = peak_result("./peakFile/GM12878_RAD21.narrowPeak")
    compare_biomaker_results(di_result_list,peak_list,5,mat_len_list)

    mat_len_list = get_mat_length('/mnt/disk1/duanran/Rao2014-GM12878-DpnII-allreps-filtered.50kb.cool')
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/GM12878')
    peak_list = peak_result("./peakFile/GM12878_RAD21.narrowPeak")
    compare_biomaker_results(is_result_list, peak_list, 5, mat_len_list)

    mat_len_list = get_mat_length('/mnt/disk1/duanran/Rao2014-GM12878-DpnII-allreps-filtered.50kb.cool')
    topdom_result_list = topdom_result_format('./topdom/result')
    peak_list = peak_result("./peakFile/GM12878_RAD21.narrowPeak")
    compare_biomaker_results(topdom_result_list, peak_list, 5, mat_len_list)

    mat_len_list = get_mat_length('/mnt/disk1/duanran/Rao2014-GM12878-DpnII-allreps-filtered.50kb.cool')
    mctad_result_list = mctad_result_format('./utad/algorithm/GM12878re',150)
    peak_list = peak_result("./peakFile/GM12878_RAD21.narrowPeak")
    compare_biomaker_results(mctad_result_list, peak_list, 5, mat_len_list)


    mat_len_list = np.loadtxt('./hunmen_gens_length', dtype=np.int)
    peak_list = peak_result("./peakFile/GM12878_SMC3.narrowPeak")
    di_result_list = di_result_format('./perl_scripts/result')
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/GM12878')
    topdom_result_list = topdom_result_format('./topdom/result')
    mctad_result_list = mctad_result_format('./utad/algorithm/GM12878re', 150)
    print('------')
    compare_biomaker_results(di_result_list, peak_list, 5, mat_len_list)
    compare_biomaker_results(is_result_list, peak_list, 5, mat_len_list)
    compare_biomaker_results(topdom_result_list, peak_list, 5, mat_len_list)
    compare_biomaker_results(mctad_result_list, peak_list, 5, mat_len_list)

    # celline = 'GM12878'
    mat_len_list = get_mat_length('/mnt/disk1/duanran/Rao2014-GM12878-DpnII-allreps-filtered.50kb.cool')
    insulation_list = load_CI_list('/mnt/disk1/duanran/{0}dense'.format(celline), celline)
    mctad_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline), 550)
    topdom_result_list = topdom_result_format('./topdom/result',celline)
    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline),celline)
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline), celline)
    EMtad_result_list = Emtad_result_format('./EMTAD_zhao_re')
    compare_celline_lunkuoxishu('/mnt/disk1/duanran/{0}dense'.format(celline), EMtad_result_list, mat_len_list,celline)
    compare_celline_lunkuoxishu('/mnt/disk1/duanran/{0}dense'.format(celline), topdom_result_list, mat_len_list, celline)
    compare_celline_lunkuoxishu('/mnt/disk1/duanran/{0}dense'.format(celline), di_result_list, mat_len_list,celline)
    compare_celline_lunkuoxishu('/mnt/disk1/duanran/{0}dense'.format(celline), is_result_list, mat_len_list, celline)

    print('run')
    insulation_list = get_insulation_score_list('./insulationscore/crane-nature-2015-master/scripts/GM12878')
    mctad_result_list = mctad_result_format('./utad/algorithm/GM12878re', 550)
    EMtad_result_list = Emtad_result_format('./EMTAD_zhao_re')
    topdom_result_list = topdom_result_format('./topdom/result','GM12878')
    di_result_list = di_result_format('./perl_scripts/GM12878result','GM12878')
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/GM12878','GM12878')
    compare_insulation_results2(EMtad_result_list, insulation_list)

    compare_insulation_results2(topdom_result_list, insulation_list)

    compare_insulation_results2(di_result_list, insulation_list)

    compare_insulation_results2(is_result_list, insulation_list)

    celline = 'GM12878'
    # get_CI_list('/mnt/disk1/duanran/{0}dense'.format(celline), celline)
    insulation_list = load_CI_list('/mnt/disk1/duanran/{0}dense'.format(celline), celline)
    mctad_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline), 150)
    EMtad_result_list = Emtad_result_format('./EMTAD_zhao_re')
    topdom_result_list = topdom_result_format('./topdom/result',celline)
    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline),celline)
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),celline)
    mc =compare_insulation_results2(EMtad_result_list, insulation_list)

    tp = compare_insulation_results2(topdom_result_list, insulation_list)

    di = compare_insulation_results2(di_result_list, insulation_list)

    iss = compare_insulation_results2(is_result_list, insulation_list)
    x = np.concatenate((mc, tp, di, iss), axis=0)
    Min = np.min(x)
    Max = np.max(x)
    x = 1 - ((x - Min) / (Max - Min))
    for vv in x:
        print(vv)


    celline = 'GM12878'
    # get_CI_list('/mnt/disk1/duanran/{0}dense'.format(celline), celline)
    CI_list = load_CI_list2('/mnt/disk1/duanran/{0}dense'.format(celline), celline)
    mctad_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline), 3000)
    topdom_result_list = topdom_result_format('./topdom/result',celline)
    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline),celline)
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),celline)

    EMtad_result_list = Emtad_result_format('./zhaoxuemin/EMTAD')

    boundary_number_list = []

    for number_index in range(len(di_result_list)):
        four_length = []
        four_length.append(len(EMtad_result_list[number_index]))
        four_length.append(len(di_result_list[number_index]))
        four_length.append(len(is_result_list[number_index]))
        four_length.append(len(topdom_result_list[number_index]))
        boundary_number_list.append(min(four_length))

    print(boundary_number_list)

    mc =compare_CI_with_top_boundary(mctad_result_list, CI_list, boundary_number_list)

    tp = compare_CI_with_top_boundary(topdom_result_list, CI_list,boundary_number_list)

    di = compare_CI_with_top_boundary(di_result_list, CI_list,boundary_number_list)

    iss = compare_CI_with_top_boundary(is_result_list, CI_list,boundary_number_list)
    x = np.concatenate((mc, tp, di, iss), axis=0)
    for vv in x:
        print(1-vv)

    celline = 'GM12878'
    all_result_list = []
    # mat_len_list = get_mat_length('/mnt/disk1/duanran/Rao2014-GM12878-DpnII-allreps-filtered.50kb.cool')
    mat_len_list = np.loadtxt('./hunmen_gens_length',dtype=np.int)
    all_result_list.append(mctad_result_format('./utad/algorithm/{0}re'.format(celline), 600))
    all_result_list.append(topdom_result_format('./topdom/result',celline))
    all_result_list.append(di_result_format('./perl_scripts/{0}result'.format(celline),celline))
    all_result_list.append(insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline), celline))
    socre_result = compare_boundary_score(all_result_list,1)

    for thod_index in range(4):
        # print('method_{0}'.format(thod_index))
        count1 = 0
        count2 = 0
        count3 = 0
        count4 = 0
        for alist in socre_result[thod_index]:
            count1 += alist.count(1)
            count2 += alist.count(2)
            count3 += alist.count(3)
            count4 += alist.count(4)
        # print('{0}\t{1}\t{2}\t{3}'.format(count1, count2, count3,count4))
        print('{0}\t{1}\t{2}\t{3}'.format(count1,count1+count2,count1+count2+count3,count1+count2+count3+count4))


    print('run')
    celline = 'NHEK'
    all_result_list = []
    # mat_len_list = get_mat_length('/mnt/disk1/duanran/Rao2014-GM12878-DpnII-allreps-filtered.50kb.cool')
    mat_len_list = np.loadtxt('./hunmen_gens_length', dtype=np.int)
    all_result_list.append(mctad_result_format('./utad/algorithm/{0}re'.format(celline), 150))
    all_result_list.append(topdom_result_format('./topdom/result', celline))
    all_result_list.append(di_result_format('./perl_scripts/{0}result'.format(celline), celline))
    all_result_list.append(
        insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),
                                 celline))
    get_venn_data(all_result_list, celline)


    celline = 'NHEK'
    # get_CI_list('/mnt/disk1/duanran/{0}dense'.format(celline), celline)
    CI_list = load_CI_list('/mnt/disk1/duanran/{0}dense'.format(celline), celline)
    scre = pd.read_excel('./venn_result/{0}_result_intersection.xlsx'.format(celline))
    compare_bdscore_marker(scre, CI_list, celline)

    print('Moc Compare')
    mat_folder = '/mnt/disk1/duanran/GM12878_low'
    celline = 'GM12878'
    cellines = ['HMEC', 'HUVEC', 'K562', 'NHEK']
    mactadre = mctad_result_format('./utad/algorithm/{0}re'.format(celline),550)
    mc_down_re = down_samping_mctad_result_format('./down_samping_re', 150, cellines)
    for index in range(22):
        gm_df = trans_to_dataframe(mactadre[index])
        down_df = trans_to_dataframe(mc_down_re['NHEK'][index])
        value = calculate_MoC_of_TAD_domain(gm_df,down_df)
        print(value)

    celline = 'GM12878'
    cellines = ['HMEC', 'HUVEC', 'K562', 'NHEK']
    topdom_result_list = topdom_result_format('./topdom/result',celline)
    topdom_down_result = down_samping_topdom_result_format('./topdom/result', cellines)
    for index in range(22):
        gm_df = trans_to_dataframe(topdom_result_list[index])
        down_df = trans_to_dataframe(topdom_down_result['NHEK'][index])
        value = calculate_MoC_of_TAD_domain(gm_df, down_df)
        print(value)

    celline = 'GM12878'
    cellines = ['HMEC', 'HUVEC', 'K562', 'NHEK']
    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline), celline)
    di_down_result = down_samping_di_result_format('./perl_scripts/result', cellines)
    for index in range(22):
        gm_df = trans_to_dataframe(di_result_list[index])
        down_df = trans_to_dataframe(di_down_result['K562'][index])
        value = calculate_MoC_of_TAD_domain(gm_df, down_df)
        print(value)


    celline = 'GM12878'
    cellines = ['HMEC', 'HUVEC', 'K562', 'NHEK']
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),celline)
    is_down_result = down_samping_insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/GM12878_low',cellines)
    for index in range(22):
        gm_df = trans_to_dataframe(is_result_list[index])
        down_df = trans_to_dataframe(is_down_result['K562'][index])
        value = calculate_MoC_of_TAD_domain(gm_df, down_df)
        print(value)

    print('run')
    celline = 'GM12878'
    cellines = ['HMEC', 'HUVEC', 'K562', 'NHEK']
    mat_len_list = get_mat_length('/mnt/disk1/duanran/Rao2014-GM12878-DpnII-allreps-filtered.50kb.cool')

    mc_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline),550)
    mc_down_result = down_samping_mctad_result_format('./down_samping_re', 550, cellines)
    topdom_result_list = topdom_result_format('./topdom/result',celline)
    topdom_down_result = down_samping_topdom_result_format('./topdom/result', cellines)
    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline), celline)
    di_down_result = down_samping_di_result_format('./perl_scripts/result', cellines)
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),celline)
    is_down_result = down_samping_insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/GM12878_low',cellines)

    low_cell = 'NHEK'

    # print('-------')
    # get_rand_index(mc_result_list, mc_down_result[low_cell])
    # print('-------')
    # get_rand_index(topdom_result_list,topdom_down_result[low_cell])
    # print('-------')
    # get_rand_index(di_result_list,di_down_result[low_cell])
    # print('-------')
    # get_rand_index(is_result_list,is_down_result[low_cell])

    get_mutual_information(mc_result_list, mc_down_result[low_cell], mat_len_list)
    get_mutual_information(topdom_result_list, topdom_down_result[low_cell], mat_len_list)
    get_mutual_information(di_result_list, di_down_result[low_cell], mat_len_list)
    get_mutual_information(is_result_list,is_down_result[low_cell],mat_len_list)

    # 25kb VS 50kb MoC
    print('25kb vs 50kb')
    print('dunranrun')
    celline = 'GM12878'
    # topdom result load
    topdom_result_list = topdom_result_format('./topdom/result',celline)
    topdom_100kb_result_list = topdom_100kb_result_format('./topdom/result',celline)
    topdom_25kb_result_list = topdom_25kb_result_format('./topdom/result', celline)
    # di result load
    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline), celline)
    di_25kb_result_list = di_25kb_result_format('./perl_scripts/result',celline)
    di_100kb_result_list = di_100kb_result_format('./perl_scripts/result', celline)
    # is result load
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),celline)
    is_25kb_result_list = insulation_25kb_result_format('./insulationscore/crane-nature-2015-master/scripts/kb25',celline)
    is_100kb_result_list = insulation_100kb_result_format('./insulationscore/crane-nature-2015-master/scripts/kb100', celline)
    # mctad result lod
    mc_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline), 550)
    mc_25kb_result_list = mctad_25kb_result_format("./utad/algorithm/{0}_25kb_re".format(celline), 300, celline)
    mc_100kb_result_list = mctad_100kb_result_format("./utad/algorithm/{0}_100kb_re".format(celline), 550,
                                                     celline)

    for index in range(len(topdom_result_list)):
        kb_25 = trans_to_dataframe(np.ceil(topdom_25kb_result_list[index]/2))
        kb_100 = trans_to_dataframe(np.ceil(topdom_100kb_result_list[index] * 2))
        kb_50 = trans_to_dataframe(topdom_result_list[index])
        value = calculate_MoC_of_TAD_domain(kb_25, kb_100)
        print(value)
    for index in range(len(mc_result_list)):
        kb_25 = trans_to_dataframe(np.ceil(mc_25kb_result_list[index]/2))
        kb_100 = trans_to_dataframe(np.ceil(mc_25kb_result_list[index] * 2))
        kb_50 = trans_to_dataframe(mc_result_list[index])
        value = calculate_MoC_of_TAD_domain(kb_25, kb_100)
        print(value)
    for index in range(len(di_result_list)):
        kb_25 = trans_to_dataframe(np.ceil(np.array(di_25kb_result_list[index])/2))
        kb_100 = trans_to_dataframe(np.ceil(np.array(di_100kb_result_list[index]) * 2))
        kb_50 = trans_to_dataframe(di_result_list[index])
        value = calculate_MoC_of_TAD_domain(kb_25, kb_100)
        print(value)
    for index in range(len(is_result_list)):
        kb_25 = trans_to_dataframe(np.ceil(np.array(is_25kb_result_list[index])/2))
        kb_100 = trans_to_dataframe(np.ceil(np.array(is_25kb_result_list[index]) * 2))
        kb_50 = trans_to_dataframe(is_result_list[index])
        value = calculate_MoC_of_TAD_domain(kb_25, kb_100)
        print(value)

    celline = 'GM12878'

    # di_down_result = down_samping_di_result_format('./perl_scripts/result', ['HMEC'])
    # is_down_result = down_samping_insulation_result_format(
    #     './insulationscore/crane-nature-2015-master/scripts/GM12878_low', ['HMEC'])

    # topdom
    topdom_result_list = topdom_result_format('./topdom/result', celline)
    topdom_down_result2 = down_samping_topdom_result_format2('./topdom/result', 'HMEC')
    topdom_down_result3 = down_samping_topdom_result_format2('./topdom/result', 'HUVEC')
    topdom_down_result4 = down_samping_topdom_result_format2('./topdom/result', 'K562')

    mc_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline), 550)
    mc_down_re2 = down_samping_mctad_result_format2('./down_samping_re', 550, 'HMEC')
    mc_down_re3 = down_samping_mctad_result_format2('./down_samping_re', 550, 'HUVEC')
    mc_down_re4 = down_samping_mctad_result_format2('./down_samping_re', 550, 'K562')

    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline), celline)
    di_down_result2 = down_samping_di_result_format2('./perl_scripts/result', 'HMEC')
    di_down_result3 = down_samping_di_result_format2('./perl_scripts/result', 'HUVEC')
    di_down_result4 = down_samping_di_result_format2('./perl_scripts/result', 'K562')

    is_result_list = insulation_result_format(
        './insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline), celline)
    is_down_result2 = down_samping_insulation_result_format2(
        './insulationscore/crane-nature-2015-master/scripts/GM12878_low', 'HMEC')
    is_down_result3 = down_samping_insulation_result_format2(
        './insulationscore/crane-nature-2015-master/scripts/GM12878_low', 'HUVEC')
    is_down_result4 = down_samping_insulation_result_format2(
        './insulationscore/crane-nature-2015-master/scripts/GM12878_low', 'K562')
    for index in range(len(topdom_result_list)):
        kb_1 = trans_to_dataframe(topdom_result_list[index])
        kb_2 = trans_to_dataframe(np.array(topdom_down_result2[index]))
        kb_3 = trans_to_dataframe(np.array(topdom_down_result3[index]))
        kb_4 = trans_to_dataframe(np.array(topdom_down_result4[index]))
        value = calculate_MoC_of_TAD_domain(kb_1, kb_4)
        print(value)

    # mctad
    for index in range(len(mc_result_list)):
        kb_1 = trans_to_dataframe(mc_result_list[index])
        kb_2 = trans_to_dataframe(np.array(mc_down_re2[index]))
        kb_3 = trans_to_dataframe(np.array(mc_down_re3[index]))
        kb_4 = trans_to_dataframe(np.array(mc_down_re4[index]))
        value = calculate_MoC_of_TAD_domain(kb_1, kb_4)
        print(value)

    # di
    for index in range(len(di_result_list)):
        kb_1 = trans_to_dataframe(di_result_list[index])
        kb_2 = trans_to_dataframe(np.array(di_down_result2[index]))
        kb_3 = trans_to_dataframe(np.array(di_down_result3[index]))
        kb_4 = trans_to_dataframe(np.array(di_down_result4[index]))
        value = calculate_MoC_of_TAD_domain(kb_1, kb_4)
        print(value)
    # is
    for index in range(len(is_result_list)):
        kb_1 = trans_to_dataframe(is_result_list[index])
        kb_2 = trans_to_dataframe(np.array(is_down_result2[index]))
        kb_3 = trans_to_dataframe(np.array(is_down_result3[index]))
        kb_4 = trans_to_dataframe(np.array(is_down_result4[index]))
        value = calculate_MoC_of_TAD_domain(kb_1, kb_4)
        print(value)
    data = pd.read_csv('./topdom/result/test_GM12878_25kb_chr2_topdomResult.domain', sep='\t')
    topre = np.ceil(np.array(data['from.id'])/2)
    data_25 = trans_to_dataframe(topre)
    data_50 = trans_to_dataframe(np.array(pd.read_csv('./topdom/result/GM128782_topdomResult.domain', sep='\t')['from.id']))
    value = calculate_MoC_of_TAD_domain(data_25, data_50)
    print(value)


    # MI 25kb Vs 50kb
    print('run')

    print('25kb vs 50kb')
    celline = 'GM12878'
    # mat_len_list = get_mat_length('/mnt/disk1/duanran/Rao2014-GM12878-DpnII-allreps-filtered.50kb.cool')
    mat_len_list = np.loadtxt('./hunmen_gens_length', dtype=np.int)


    mc_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline), 550)
    mc_100kb_result_list = mctad_100kb_result_format("./utad/algorithm/{0}_100kb_re".format(celline), 550,celline)
    mc_25kb_result_list = mctad_25kb_result_format("./utad/algorithm/{0}_25kb_re".format(celline), 550,
                                                   celline)
    for index in range(len(mc_result_list)):

        re = np.array(np.ceil(np.array(mc_100kb_result_list[index])*2), dtype=np.int)
        kb_100 = MI_list_to_labes(re, mat_len_list[index])
        kb_25 = MI_list_to_labes(np.array(np.ceil(mc_25kb_result_list[index] / 2),dtype=np.int),mat_len_list[index])
        kb_50 = MI_list_to_labes(mc_result_list[index], mat_len_list[index])
        value = metrics.mutual_info_score(kb_100, kb_25)
        print(value)


    topdom_result_list = topdom_result_format('./topdom/result',celline)
    topdom_100kb_result_list = topdom_100kb_result_format('./topdom/result',celline)
    topdom_25kb_result_list = topdom_25kb_result_format('./topdom/result', celline)


    for index in range(len(topdom_result_list)):
        kb_100 = MI_list_to_labes(np.array(np.ceil(np.array(topdom_100kb_result_list[index]) * 2), dtype=np.int),
                                  mat_len_list[index])
        kb_25 = MI_list_to_labes(np.array(np.ceil(np.array(topdom_25kb_result_list[index]) / 2), dtype=np.int),
                                  mat_len_list[index])
        kb_50 = MI_list_to_labes(topdom_result_list[index], mat_len_list[index])
        value = metrics.mutual_info_score(kb_25, kb_100)
        print(value)

    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline), celline)
    di_25kb_result_list = di_25kb_result_format('./perl_scripts/result',celline)
    di_100kb_result_list = di_100kb_result_format('./perl_scripts/result', celline)
    for index in range(len(di_result_list)):
        mat_len = mat_len_list[index]

        kb_25 = MI_list_to_labes(np.array(np.ceil(np.array(di_25kb_result_list[index]) / 2), dtype=np.int), mat_len)
        kb_100 = MI_list_to_labes(np.array(np.array(di_100kb_result_list[index]) * 2, dtype=np.int), mat_len)
        kb_50 = MI_list_to_labes(np.array(di_result_list[index], dtype=np.int),mat_len)
        value = metrics.mutual_info_score(kb_50, kb_100)
        print(value)


    celline = 'NHEK'
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),celline)
    is_25kb_result_list = insulation_25kb_result_format('./insulationscore/crane-nature-2015-master/scripts/kb25',celline)
    is_100kb_result_list = insulation_100kb_result_format('./insulationscore/crane-nature-2015-master/scripts/kb100', celline)
    for index in range(len(is_result_list)):
        mat_len = mat_len_list[index]
        kb_25 = MI_list_to_labes(np.array(np.ceil(np.array(is_25kb_result_list[index]) / 2), dtype=np.int), mat_len)

        kb_100 = MI_list_to_labes(np.array(np.ceil(np.array(is_100kb_result_list[index]) * 2), dtype=np.int), mat_len)
        kb_50 = MI_list_to_labes(np.array(is_result_list[index], dtype=np.int),mat_len)
        value = metrics.mutual_info_score(kb_25, kb_100)
        print(value)


    mc_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline), 550)
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),celline)
    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline), celline)
    topdom_result_list = topdom_result_format('./topdom/result',celline)

    sum = 0
    for index in range(len(mc_result_list)):
        mat_len = mat_len_list[index]

        kb_mc = MI_list_to_labes(np.array(di_result_list[index], dtype=np.int),mat_len)
        kb_top = MI_list_to_labes(np.array(is_result_list[index], dtype=np.int), mat_len)
        value = metrics.mutual_info_score(kb_mc, kb_top)
        sum +=value
    print(sum/len(mc_result_list))

    mc_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline), 300)
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),celline)
    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline), celline)
    topdom_result_list = topdom_result_format('./topdom/result',celline)

    sum = 0
    for index in range(len(mc_result_list)):
        mat_len = mat_len_list[index]

    kb_mc = trans_to_dataframe(mc_result_list[index])
        kb_top = trans_to_dataframe(di_result_list[index])
        value = calculate_MoC_of_TAD_domain(kb_mc, kb_top)
        sum +=value
    print(sum/len(mc_result_list))

    print('run')
    celline = ['GM12878','HMEC','HUVEC','K562',"NHEK"]
    celline = 'GM12878'
    mat_len_list = np.loadtxt('./hunmen_gens_length', dtype=np.int)
    mc_result_list = mctad_result_format('./utad/algorithm/{0}re'.format(celline), 550)
    is_result_list = insulation_result_format('./insulationscore/crane-nature-2015-master/scripts/{0}'.format(celline),celline)
    di_result_list = di_result_format('./perl_scripts/{0}result'.format(celline), celline)
    top_result_list = topdom_result_format('./topdom/result',celline)

    ctcf_path = './histonepeakfile/wgEncodeAwgTfbsBroad{0}CtcfUniPk.narrowPeak'.format(celline)
        chrnum = 2
        mat_len = 4864*10
        resulution = 10000
        scale = 50
        ctct_list = get_CTCF_makerlist(ctcf_path, chrnum ,mat_len,resulution)
    print(len(ctct_list))

    cmm = is_result_list[1]
    new_loc = []
    for boundary in cmm:
        new_loc.append((boundary-1)*5+3)
    new_loc = np.array(new_loc,dtype=np.int)

    fin_list = np.array([0]*101)
    for boundary in new_loc:
        if boundary - scale > 0 and boundary + scale < mat_len+1000:

            print(len(ctct_list[boundary - scale:boundary + scale + 1]))
            fin_list = fin_list + ctct_list[boundary-scale:boundary+scale+1]
    fin_list = fin_list / len(new_loc)

    resulution = 50000
    fin_list = np.array([0] * 21)
    lens = 0
    for chrnum in range(1,23):
        mat_len = mat_len_list[chrnum-1]
        scale = 10
        ctct_list = get_CTCF_makerlist(ctcf_path, chrnum ,mat_len,resulution)
        cmm = mc_result_list[chrnum-1]
        lens += len(cmm)
        for boundary in cmm:
            if boundary - scale > 0 and boundary + scale < mat_len:
                fin_list = fin_list + ctct_list[boundary-scale:boundary+scale+1]

    avg_len_re = fin_list/lens
    for vv in avg_len_re:
        print(vv)
    print(lens)
    center_value = max(fin_list)
    background_value = (np.mean(fin_list[0:5]) + np.mean(fin_list[15:20]))/2
    fold_change = center_value/background_value
    print(fold_change)
    for vv in fin_list:
        print(vv)